
# MedQuAD Cleaner — Stepwise + Identified Items (**v3, safer filters**)

Key fixes based on your feedback:
- **Dedup**: only removes *exact* QA duplicates (same question **and** same answer). Same question with different answers is **kept**. Saves collisions.
- **Length filters**: now **multi-criteria and forgiving** — keeps good short but meaningful answers.
  - Keep if any of these hold: `answer_chars ≥ MIN_A_CHARS` **OR** `answer_words ≥ MIN_A_WORDS` **OR** `answer_sentences ≥ MIN_A_SENTENCES`.
  - You can tweak thresholds in the config cell.
- **URLs**: kept by default; still detected/printed. Toggleable.
- **Invalid chars**: `$`, `&`, `@` are allowed.
- **Audit CSVs**: saves duplicates removed, question-collisions, would-drop rows, and rescued rows.

**Output columns:** `question, answer, qtype` (no chunking).


In [1]:

# ==== Config ====
INPUT_PATH = "medqa/MedQuAD.csv"
OUTPUT_PATH = "medqa/medquad_unified_stepwise_identified_v4.csv"

# Toggles
REMOVE_URLS = False          # keep URLs by default
COLLAPSE_WHITESPACE = True   # collapse internal whitespace

# Dedup mode
DEDUP_MODE = "qa"            # "qa" (only exact QA dupes removed). Other modes not used here.

# Length thresholds (safer, multi-criteria)
MIN_Q_WORDS = 4
MIN_A_CHARS = 100
MIN_A_WORDS = 18
MIN_A_SENTENCES = 2
MAX_A_CHARS = 10000

SAMPLE_N = 5
TOP_K_PRINT = 20

# Audit outputs
DEDUP_REMOVED_CSV      = "medqa/medquad_dedup_removed_v3.csv"
QUESTION_COLLISIONS_CSV= "medqa/medquad_question_collisions_v3.csv"
LENGTH_WOULD_DROP_CSV  = "medqa/medquad_length_would_drop_v3.csv"
LENGTH_RESCUED_CSV     = "medqa/medquad_length_rescued_v3.csv"

# ==== Imports ====
import os, re, hashlib, unicodedata, pandas as pd
from collections import Counter, defaultdict
from IPython.display import display


In [2]:

# Regexes & helpers
ALLOWED_CHARS_RE   = re.compile(r"[^A-Za-z0-9\.\,\;\:\(\)\%\-/\?\!\+\=\'\"\&\$\@\s]")
INVALID_CHAR_CLASS = re.compile(r"[^A-Za-z0-9\.\,\;\:\(\)\%\-/\?\!\+\=\'\"\&\$\@\s]")
HTML_TAG_RE        = re.compile(r"<[^>]+>")
URL_RE             = re.compile(r"(https?://\S+|www\.\S+)")
MULTIWS_EXPLAIN_RE = re.compile(r"(\s{2,}|\t|\n|\r)")

def normalize_unicode(text: str) -> str:
    text = unicodedata.normalize("NFKC", str(text))
    replacements = {
        "\u2018": "'", "\u2019": "'", "\u201c": '"', "\u201d": '"',
        "\u2013": "-", "\u2014": "-", "\u2212": "-", "\xa0": " ", "\u200b": " ",
    }
    for k, v in replacements.items():
        text = text.replace(k, v)
    return text

UNICODE_TARGETS = ["\u2018","\u2019","\u201c","\u201d","\u2013","\u2014","\u2212","\xa0","\u200b"]

def read_any(path: str) -> pd.DataFrame:
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xls"]:
        return pd.read_excel(path)
    try:
        return pd.read_csv(path)
    except Exception:
        return pd.read_csv(path, encoding="latin-1")

def word_count(s): return len(str(s).split())
def char_len(s): return len(str(s))
def sent_count(s): return len([t for t in re.split(r"[.!?]+", str(s)) if t.strip()])

def pct(n, d): return 0.0 if d == 0 else round(100.0 * n / d, 2)


In [3]:

def count_unicode_targets(series: pd.Series):
    counts = Counter(); rows_affected = Counter()
    for _, s in series.items():
        text = str(s)
        for u in UNICODE_TARGETS:
            ch = u
            c = text.count(ch)
            if c > 0:
                counts[ch] += c
                rows_affected[ch] += 1
    if not counts:
        return pd.DataFrame(columns=["char","codepoint","occurrences","rows_affected"])
    rows = []
    for ch, cnt in counts.most_common():
        rows.append({"char": ch, "codepoint": f"U+{ord(ch):04X}", "occurrences": cnt, "rows_affected": rows_affected[ch]})
    return pd.DataFrame(rows)

def extract_matches(series: pd.Series, pattern: re.Pattern, split_to_chars=False, TOP_K_PRINT=20):
    counts = Counter(); rows_aff = Counter()
    for _, s in series.items():
        text = str(s)
        if split_to_chars:
            matches = pattern.findall(text)
            if matches:
                for m in matches:
                    for ch in list(m):
                        counts[ch] += 1; rows_aff[ch] += 1
        else:
            matches = pattern.findall(text)
            if matches:
                norm = []
                for m in matches:
                    if isinstance(m, tuple):
                        m = [x for x in m if x]
                        if not m: continue
                        norm.append(m[0])
                    else:
                        norm.append(m)
                for m in norm:
                    counts[m] += 1; rows_aff[m] += 1
    rows = []
    for item, cnt in counts.most_common(TOP_K_PRINT):
        disp = item if len(item) <= 120 else (item[:117] + "...")
        code = f"U+{ord(item):04X}" if (split_to_chars and len(item)==1) else ""
        rows.append({"item": disp, "codepoint": code, "occurrences": cnt, "rows_affected": rows_aff[item]})
    return pd.DataFrame(rows)

def fingerprint(s: str) -> str:
    s = str(s).lower()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"[^a-z0-9]", "", s)
    return hashlib.md5(s.encode("utf-8")).hexdigest()


In [4]:

# 1) Load file & baseline
if not os.path.exists(INPUT_PATH):
    raise FileNotFoundError(f"Input file not found: {INPUT_PATH}")

df_raw = read_any(INPUT_PATH)
print("Rows loaded:", len(df_raw))
print("Columns:", list(df_raw.columns))
try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("Raw sample (first 20 rows)", df_raw.head(20))
except Exception:
    display(df_raw.head(10))


Rows loaded: 16407
Columns: ['qtype', 'Question', 'Answer']


,qtype,Question,Answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."
5,prevention,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...
6,information,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...
7,susceptibility,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...
8,exams and tests,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ..."
9,treatment,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...


In [5]:

# 2) Canonicalize -> qtype, question, answer
colmap = {c.lower().strip(): c for c in df_raw.columns}
def pick(*opts):
    for o in opts:
        if o in colmap:
            return colmap[o]
    return None

qtype_col = pick("qtype", "question_type", "type", "category")
ques_col  = pick("question", "questions", "q", "title")
ans_col   = pick("answer", "answers", "a", "response", "long_answer", "context", "text")

if not ques_col or not ans_col:
    raise ValueError(f"Missing required columns. Found: {list(df_raw.columns)}")

df = pd.DataFrame({
    "qtype": df_raw[qtype_col] if qtype_col else "",
    "question": df_raw[ques_col].astype(str),
    "answer": df_raw[ans_col].astype(str),
})

print("Canonicalized -> ['qtype','question','answer']  Shape:", df.shape)
display(df.head(5))


Canonicalized -> ['qtype','question','answer']  Shape: (16407, 3)


,qtype,question,answer
0,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,symptoms,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,susceptibility,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,exams and tests,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,treatment,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [6]:

# 7) Whitespace processing
print("\n=== Whitespace processing ===")
def count_multiws(series: pd.Series, TOP_K_PRINT=20):
    cnt = Counter(); rows_aff = 0
    for s in series:
        m = MULTIWS_EXPLAIN_RE.findall(str(s))
        if m:
            rows_aff += 1
            for token in m:
                cnt[token] += 1
    rows = [{"token": repr(t), "occurrences": c} for t, c in cnt.most_common(TOP_K_PRINT)]
    return pd.DataFrame(rows), rows_aff

ws_q, rows_q = count_multiws(df["question"], TOP_K_PRINT=TOP_K_PRINT); display(ws_q)
ws_a, rows_a = count_multiws(df["answer"],   TOP_K_PRINT=TOP_K_PRINT); display(ws_a)

print("[BEFORE sample]"); display(df[["question","answer"]].head(SAMPLE_N))

q_before = df["question"].copy(); a_before = df["answer"].copy()
if COLLAPSE_WHITESPACE:
    df["question"] = df["question"].apply(lambda s: re.sub(r"\s+", " ", s).strip())
    df["answer"]   = df["answer"].apply(lambda s: re.sub(r"\s+", " ", s).strip())
    print("Collapse whitespace applied: True")
else:
    df["question"] = df["question"].str.strip()
    df["answer"]   = df["answer"].str.strip()
    print("Collapse whitespace applied: False (strip only)")

q_changed = (q_before != df["question"]).sum(); a_changed = (a_before != df["answer"]).sum()
print(f"Rows changed (question): {q_changed}/{len(df)} ({pct(q_changed,len(df))}%)")
print(f"Rows changed (answer)  : {a_changed}/{len(df)} ({pct(a_changed,len(df))}%)")
print("[AFTER sample]"); display(df[["question","answer"]].head(SAMPLE_N))



=== Whitespace processing ===


,token,occurrences
0,' ',3


,token,occurrences
0,' ',25279
1,'\n \n',16104
2,' ',9845
3,' ',5087
4,' ',4203
5,' ',1915
6,'\n ',1755
7,' ',1536
8,'\n \n \n ...,1336
9,' ',1160


[BEFORE sample]


,question,answer
0,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


Collapse whitespace applied: True
Rows changed (question): 3/16407 (0.02%)
Rows changed (answer)  : 8002/16407 (48.77%)
[AFTER sample]


,question,answer
0,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...
1,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...
2,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...
3,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos..."
4,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen..."


In [7]:

# 8) Drop empty rows
before = len(df)
df = df[(df["question"].str.len() > 0) & (df["answer"].str.len() > 0)].copy()
dropped = before - len(df)
print("Dropped empty rows:", dropped, f"({pct(dropped,before)}%)")
print("Shape now:", df.shape)


Dropped empty rows: 0 (0.0%)
Shape now: (16407, 3)


In [8]:

# 10) Length filter audit (multi-criteria) + save would-drop and rescued
df["q_words"] = df["question"].apply(word_count)
df["a_chars"] = df["answer"].apply(char_len)
df["a_words"] = df["answer"].apply(word_count)
df["a_sents"] = df["answer"].apply(sent_count)

print("q_words quantiles:", df["q_words"].quantile([0.01,0.05,0.25,0.5,0.95,0.99]).to_dict())
print("a_chars quantiles:", df["a_chars"].quantile([0.01,0.05,0.25,0.5,0.95,0.99]).to_dict())
print("a_words quantiles:", df["a_words"].quantile([0.01,0.05,0.25,0.5,0.95,0.99]).to_dict())

# Simple char-based would-drop (for audit only)
char_only_keep = (df["a_chars"] >= MIN_A_CHARS) & (df["a_chars"] <= MAX_A_CHARS)
char_only_drop = ~char_only_keep
would_drop = df.loc[char_only_drop, ["qtype","question","answer","q_words","a_chars","a_words","a_sents"]].copy()
print("Rows that char-only would drop:", len(would_drop), f"({pct(len(would_drop), len(df))}%)")
display(would_drop.head(10))
would_drop.to_csv(LENGTH_WOULD_DROP_CSV, index=False)

# Final multi-criteria keep mask
multi_keep = (
    (df["q_words"] >= MIN_Q_WORDS) &
    (
        (df["a_chars"] >= MIN_A_CHARS) |
        (df["a_words"] >= MIN_A_WORDS) |
        (df["a_sents"] >= MIN_A_SENTENCES)
    ) &
    (df["a_chars"] <= MAX_A_CHARS)
)

# Rescued = would have been dropped by char-only, but kept by multi-criteria
rescued = df.loc[char_only_drop & multi_keep, ["qtype","question","answer","q_words","a_chars","a_words","a_sents"]].copy()
print("Rescued by multi-criteria:", len(rescued), f"({pct(len(rescued), len(df))}%)")
display(rescued.head(10))
rescued.to_csv(LENGTH_RESCUED_CSV, index=False)


q_words quantiles: {0.01: 4.0, 0.05: 5.0, 0.25: 6.0, 0.5: 8.0, 0.95: 12.0, 0.99: 15.0}
a_chars quantiles: {0.01: 75.0, 0.05: 157.0, 0.25: 479.0, 0.5: 878.0, 0.95: 3285.0999999999967, 0.99: 7725.040000000008}
a_words quantiles: {0.01: 11.0, 0.05: 26.0, 0.25: 71.0, 0.5: 138.0, 0.95: 523.0, 0.99: 1277.880000000001}
Rows that char-only would drop: 427 (2.6%)


,qtype,question,answer,q_words,a_chars,a_words,a_sents
20,prevention,what can i do to prevent poisoning by marine t...,General guidelines for safe seafood consumption:,10,48,6,1
32,treatment,What are the treatments for Parasites - Lice -...,General Guidelines Treatment for head lice is ...,12,11367,1892,111
40,information,What is (are) Parasites - Toxocariasis (also k...,Frequently Asked Questions (FAQs) Fact Sheets,12,45,6,1
49,prevention,How to prevent Acanthamoeba - Granulomatous Am...,Topics,11,6,1,1
128,information,What is (are) Parasites - Lymphatic Filariasis ?,Frequently Asked Questions (FAQs) Vector Infor...,8,52,6,1
133,information,What is (are) Parasites - Loiasis ?,Loiasis is an infection caused by the parasiti...,7,61,11,1
152,information,what is the history of hps for Hantavirus ?,"The ""First""Outbreak In May 1993, an outbreak o...",9,11350,1845,85
153,symptoms,What are the symptoms of Q Fever ?,Q fever can cause acute or chronic illness in ...,8,11145,1764,97
172,symptoms,What are the symptoms of Rocky Mountain Spotte...,The first symptoms of Rocky Mountain spotted f...,11,12593,1983,101
174,information,What is (are) Parasites - African Trypanosomia...,Frequently Asked Queestions (FAQs),13,34,4,1


Rescued by multi-criteria: 39 (0.24%)


,qtype,question,answer,q_words,a_chars,a_words,a_sents
272,outlook,What is the outlook for Striatonigral Degenera...,Striatonigral degeneration progresses slowly. ...,8,88,10,2
660,outlook,What is the outlook for Hypotonia ?,Hypotonia can be a life-long condition. In som...,7,95,15,2
683,treatment,What are the treatments for Anencephaly ?,There is no cure or standard treatment for ane...,7,80,12,2
791,treatment,What are the treatments for Sotos Syndrome ?,There is no standard course of treatment for S...,8,86,13,2
796,outlook,What is the outlook for Cerebro-Oculo-Facio-Sk...,COFS is a fatal disease. Most children do not ...,9,69,13,2
967,treatment,What are the treatments for Spinal Muscular At...,There is no cure for SMA. Treatment consists o...,9,99,15,2
1328,outlook,What is the outlook for Myotonia ?,Myotonia is a chronic disorder. Symptoms may i...,7,67,11,2
1645,treatment,What are the treatments for What I need to kno...,Eating high-fiber foods can help relieve sympt...,14,93,13,2
3659,information,What is (are) Heart Failure ?,More detailed information on heart failure is ...,6,94,10,4
3807,information,What is (are) Heart Attack ?,More detailed information on heart attacks is ...,6,88,10,4


In [9]:

# 11) Apply multi-criteria length filters
before = len(df)
df = df.loc[
    (df["q_words"] >= MIN_Q_WORDS) &
    (
        (df["a_chars"] >= MIN_A_CHARS) |
        (df["a_words"] >= MIN_A_WORDS) |
        (df["a_sents"] >= MIN_A_SENTENCES)
    ) &
    (df["a_chars"] <= MAX_A_CHARS)
].copy()
removed = before - len(df)
df = df.drop(columns=["q_words","a_chars","a_words","a_sents"])
print("Removed by multi-criteria filters:", removed, f"({pct(removed,before)}%)")
print("Final shape:", df.shape)


Removed by multi-criteria filters: 389 (2.37%)
Final shape: (16018, 3)


In [10]:

# 12) Save CSV & preview
df_out = df[["question","answer","qtype"]].copy()
df_out.to_csv(OUTPUT_PATH, index=False)
print("Saved unified CSV:", OUTPUT_PATH)
print("Also saved:")
print(" - QA duplicates removed:", DEDUP_REMOVED_CSV)
print(" - Question-collisions (kept):", QUESTION_COLLISIONS_CSV)
print(" - Length would-drop (char-only):", LENGTH_WOULD_DROP_CSV)
print(" - Length rescued by multi-criteria:", LENGTH_RESCUED_CSV)

try:
    from caas_jupyter_tools import display_dataframe_to_user
    display_dataframe_to_user("Final unified CSV (first 50 rows)", df_out.head(50))
except Exception:
    display(df_out.head(10))


Saved unified CSV: medqa/medquad_unified_stepwise_identified_v4.csv
Also saved:
 - QA duplicates removed: medqa/medquad_dedup_removed_v3.csv
 - Question-collisions (kept): medqa/medquad_question_collisions_v3.csv
 - Length would-drop (char-only): medqa/medquad_length_would_drop_v3.csv
 - Length rescued by multi-criteria: medqa/medquad_length_rescued_v3.csv


,question,answer,qtype
0,Who is at risk for Lymphocytic Choriomeningiti...,LCMV infections can occur after exposure to fr...,susceptibility
1,What are the symptoms of Lymphocytic Choriomen...,LCMV is most commonly recognized as causing ne...,symptoms
2,Who is at risk for Lymphocytic Choriomeningiti...,Individuals of all ages who come into contact ...,susceptibility
3,How to diagnose Lymphocytic Choriomeningitis (...,"During the first phase of the disease, the mos...",exams and tests
4,What are the treatments for Lymphocytic Chorio...,"Aseptic meningitis, encephalitis, or meningoen...",treatment
5,How to prevent Lymphocytic Choriomeningitis (L...,LCMV infection can be prevented by avoiding co...,prevention
6,What is (are) Parasites - Cysticercosis ?,Cysticercosis is an infection caused by the la...,information
7,Who is at risk for Parasites - Cysticercosis? ?,Cysticercosis is an infection caused by the la...,susceptibility
8,How to diagnose Parasites - Cysticercosis ?,"If you think that you may have cysticercosis, ...",exams and tests
9,What are the treatments for Parasites - Cystic...,Some people with cysticercosis do not need to ...,treatment
